<a href="https://colab.research.google.com/github/FouadAIAzar/Flurine/blob/main/get_eDGAR_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get a Database, like eDGAR

In [20]:
# Importing the required modules
import os
import sys
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [2]:
# Get Table from eDGAR
URL = "http://edgar.biocomp.unibo.it/cgi-bin/gene_disease_db/main_table.py"
  
# sending get request and saving the response as r
r = requests.get(url = URL)
with open('./response.html', 'wb+') as f:
    f.write(r.content)

In [5]:
## Convert HTML table to .csv

path = 'response.html'
  
# Create an empty list
data = []

# the HTML file
header = [] # <th></th> attribute
sp = BeautifulSoup(open(path),'html.parser')
tableRow = sp.find_all("table")[0].find("tr") # find table rows
 
for i in tableRow:
    try:
        header.append(i.get_text())
    except:
        continue
 
# for getting the data
HTML_data = sp.find_all("table")[0].find_all("tr")[1:]
 
for i in HTML_data:
    sub = []
    for j in i:
        try:
            sub.append(j.get_text())
        except:
            continue
    data.append(sub)

# Store the results in pd's DataFrame for SQL processing
df = pd.DataFrame(data = data, columns = header)
  
# convert into DataFrame into CSV
df.to_csv('disease2gene.csv')

# Look up a disease in eDGAR

In [23]:
term = "KERATOCONUS"
rows = df[df['Disease Name'].str.contains(term)]
if rows.empty:
  print("No genes found")
else:
  print(rows)

genes = row.iloc[:,0].values # array of gene asociated

print(genes)

    Gene Name   Disease Name  Disease ID                   Database
1       VSX1    KERATOCONUS    PS148300    ClinVar, OMIM, HUMSAVAR 
593   ZNF469    KERATOCONUS    PS148300                    ClinVar 
[' VSX1 ' ' ZNF469 ']


# Query FASTA gene sequence from NCBI

In [77]:
import requests
import json
from pandas import json_normalize

def fetch_gene_seq(genes: str) -> list:
  results = pd.DataFrame()  
  for g in genes:
    url=f"https://clinicaltables.nlm.nih.gov/api/ncbi_genes/v3/search?terms={genes}"
    response = json_normalize(requests.request("GET",url))
    results.append(response)
    return results



In [78]:
test = fetch_gene_seq(genes)

print(test)

Empty DataFrame
Columns: []
Index: []


In [76]:
url=f"https://clinicaltables.nlm.nih.gov/api/ncbi_genes/v3/search?terms=FBLN5"
response = requests.request("GET",url)
print(response.json())

[1, ['HGNC:3602'], None, [['HGNC-Symb', 'HGNC:3602', '14', 'FBLN5', 'fibulin 5', 'protein-coding']], ['HGNC-Symb']]
